# SoftmaxClassifier_fancy
이전에는 cost를 구할 때, 아래의 단계를 거쳐서 cross_entropy를 직접 계산하였다.
- logits = tf.matmul(X, W) + b
- hypothesis = tf.nn.softmax(logits)
- cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))

여기에서는 softmax_corss_entropy_with_logits_v2 함수를 이용하여, logits을 받아서 cross_entropy를 바로 구한다.

In [2]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, -1]

# preview
print(x_data.shape, y_data.shape)
print(x_data[:10])
print(y_data[:10])

(101, 16) (101,)
[[1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 4. 0. 0. 1.]
 [1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 4. 1. 0. 1.]
 [0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 4. 0. 0. 1.]
 [1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 4. 1. 0. 1.]
 [1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 4. 1. 0. 1.]
 [1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 4. 1. 1. 1.]
 [0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0.]
 [0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 4. 0. 1. 0.]]
[0. 0. 3. 0. 0. 0. 0. 3. 3. 0.]


y_data가 integer label로 되어 있기 때문에, one-hot 인코딩으로 바꿔어 준다.

In [3]:
n_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None])  # 0 ~ 6
Y_one_hot = tf.one_hot(Y, n_classes)  # one hot
print(Y_one_hot)

Tensor("one_hot:0", shape=(?, 7), dtype=float32)


In [4]:
W = tf.Variable(tf.random_normal([16, n_classes]), name='weight')
b = tf.Variable(tf.random_normal([n_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))


Step:     0	Loss: 4.117	Acc: 4.95%
Step:   200	Loss: 0.383	Acc: 89.11%
Step:   400	Loss: 0.237	Acc: 96.04%
Step:   600	Loss: 0.167	Acc: 97.03%
Step:   800	Loss: 0.128	Acc: 99.01%
Step:  1000	Loss: 0.104	Acc: 100.00%
Step:  1200	Loss: 0.087	Acc: 100.00%
Step:  1400	Loss: 0.075	Acc: 100.00%
Step:  1600	Loss: 0.067	Acc: 100.00%
Step:  1800	Loss: 0.060	Acc: 100.00%
Step:  2000	Loss: 0.054	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 3 True Y: 3
[True] Prediction: 6 True Y: 6
[True] Prediction: 6 True Y: 6
[True] Prediction: 6 True Y: 6
[True] Prediction: 1 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Pre